In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

from keras import Sequential,losses
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.optimizers import SGD
from keras.utils import np_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv
/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data/A_Z Handwritten Data.csv


In [5]:
df=pd.read_csv("/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv")

In [6]:
df.drop_duplicates(inplace=True)

In [17]:
datagen = ImageDataGenerator(width_shift_range=2,height_shift_range=2,validation_split=0.2)

In [18]:
X = df.loc[:,"0.1":]
y = df.loc[:,"0"]

In [21]:
X=X.values.reshape(len(X),28,28,1)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.85)
X_train = X_train/255
X_test = X_test/255
y_train=np_utils.to_categorical(y_train)

In [22]:
model=Sequential()  
model.add(Conv2D(16,(3,3),activation="relu"))
model.add(MaxPooling2D(strides=(1,1)))
model.add(Conv2D(8,(3,3),activation="relu"))
model.add(MaxPooling2D(strides=(1,1)))
model.add(Conv2D(8,(3,3),activation="relu"))
model.add(MaxPooling2D(strides=(1,1)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(26,activation="softmax"))
model.compile(loss=losses.categorical_crossentropy,metrics=["accuracy"],optimizer=SGD(learning_rate=0.05))

In [23]:
checkpoint = ModelCheckpoint(filepath="model_sgd.pkl",monitor="val_accuracy",verbose=2,save_best_only=True,mode="max")

In [24]:
model.fit(datagen.flow(X_train, y_train, batch_size=128,subset="training"),
          validation_data=datagen.flow(X_train, y_train, batch_size=128,subset="validation"),
          epochs=50,batch_size=128,verbose=2,callbacks=[checkpoint])

Epoch 1/50

Epoch 1: val_accuracy improved from -inf to 0.95320, saving model to model_sgd.pkl
1069/1069 - 54s - loss: 0.5879 - accuracy: 0.8325 - val_loss: 0.1641 - val_accuracy: 0.9532 - 54s/epoch - 50ms/step
Epoch 2/50

Epoch 2: val_accuracy improved from 0.95320 to 0.96958, saving model to model_sgd.pkl
1069/1069 - 52s - loss: 0.1377 - accuracy: 0.9608 - val_loss: 0.1105 - val_accuracy: 0.9696 - 52s/epoch - 49ms/step
Epoch 3/50

Epoch 3: val_accuracy improved from 0.96958 to 0.97221, saving model to model_sgd.pkl
1069/1069 - 53s - loss: 0.1026 - accuracy: 0.9708 - val_loss: 0.0967 - val_accuracy: 0.9722 - 53s/epoch - 49ms/step
Epoch 4/50

Epoch 4: val_accuracy improved from 0.97221 to 0.97686, saving model to model_sgd.pkl
1069/1069 - 53s - loss: 0.0883 - accuracy: 0.9752 - val_loss: 0.0826 - val_accuracy: 0.9769 - 53s/epoch - 50ms/step
Epoch 5/50

Epoch 5: val_accuracy improved from 0.97686 to 0.97774, saving model to model_sgd.pkl
1069/1069 - 52s - loss: 0.0769 - accuracy: 0.9784

In [25]:
y_predict=model.predict(X_test)
y_predict = np.argmax(y_predict,axis=1)
accuracy_score(y_predict,y_test)

943/943 [==============================] - 2s 2ms/step


0.9854798607657882

In [26]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1090
           1       0.98      0.99      0.99       710
           2       0.99      0.98      0.99      1819
           3       0.95      0.97      0.96       813
           4       0.98      0.99      0.98       926
           5       0.97      1.00      0.99       182
           6       0.96      0.96      0.96       527
           7       0.99      0.91      0.95       639
           8       0.97      0.99      0.98       160
           9       0.98      0.96      0.97       714
          10       0.96      0.98      0.97       488
          11       0.99      0.98      0.99       901
          12       0.99      0.99      0.99      1478
          13       0.96      1.00      0.98      1454
          14       0.99      0.99      0.99      4383
          15       0.99      0.99      0.99      1513
          16       0.97      0.98      0.97       512
          17       0.99    

In [27]:
import shutil
shutil.make_archive("model_sgd.pkl", 'zip', "/kaggle/working/model_sgd.pkl")

'/kaggle/working/model_sgd.pkl.zip'

In [ ]:
pd.to_pickle(model,"cnn_p.pkl")

In [29]:
from keras.models import load_model
model1=load_model("/kaggle/working/model_sgd.pkl")

In [30]:
y_predict=model1.predict(X_test)
y_predict = np.argmax(y_predict,axis=1)
accuracy_score(y_predict,y_test)

943/943 [==============================] - 2s 2ms/step


0.9866401458644124

In [31]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1100
           1       0.98      0.98      0.98       711
           2       0.99      0.99      0.99      1807
           3       0.94      0.96      0.95       815
           4       0.98      0.99      0.98       928
           5       0.98      1.00      0.99       183
           6       0.95      0.99      0.97       505
           7       0.97      0.98      0.98       583
           8       0.98      0.98      0.98       163
           9       0.97      0.98      0.98       698
          10       0.97      0.98      0.97       493
          11       0.99      0.98      0.98       910
          12       1.00      0.99      0.99      1484
          13       0.99      0.99      0.99      1507
          14       0.99      0.99      0.99      4367
          15       0.99      0.98      0.99      1527
          16       0.97      0.96      0.97       521
          17       0.99    